In [30]:
#Current working directory
import os
print(os.getcwd())

C:\Users\Aryan Paritosh\AppData\Local\Programs\Microsoft VS Code


In [33]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Myntra Search URL for Purses (First Page) sorted by popularity
myntra_url = "https://www.myntra.com/purses?sort=popularity"

# Set Chrome Options (Headless Mode)
options = Options()
options.add_argument("--headless")  # Run in the background
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0")

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(myntra_url)

# Wait for products to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-base")))

# Extract Purse Listings
purses = []

# Loop through multiple pages
page = 1
while True:
    print(f"📄 Scraping Page {page}...")
    
    # Extract product elements
    items = driver.find_elements(By.CLASS_NAME, "product-base")
    product_links = []

    for item in items:
        try:
            product_link = item.find_element(By.TAG_NAME, "a").get_attribute("href")
            product_links.append(product_link)
        except:
            continue  # Skip if link is missing

    # Visit each product page and extract details
    for link in product_links:
        try:
            print(f"🔗 Opening Product: {link}")
            driver.get(link)
            time.sleep(3)  # Allow page to load

            product_soup = BeautifulSoup(driver.page_source, "html.parser")

            # Extract Product Details
            brand = product_soup.find("h1", class_="pdp-title").text.strip() if product_soup.find("h1", class_="pdp-title") else "N/A"
            name = product_soup.find("h1", class_="pdp-name").text.strip() if product_soup.find("h1", class_="pdp-name") else "N/A"
            discounted_price = product_soup.find("span", class_="pdp-price").text.strip() if product_soup.find("span", class_="pdp-price") else "N/A"
            original_price = product_soup.find("span", class_="pdp-mrp").text.strip() if product_soup.find("span", class_="pdp-mrp") else "N/A"

            rating = product_soup.find("div", class_="index-overallRating")
            review_count = product_soup.find("div", class_="index-ratingsCount")
            material_info = product_soup.find("div", class_="index-productDescription")
            stock_info = product_soup.find("div", class_="pdp-price-info")
            seller_info = product_soup.find("div", class_="supplier-name")

            # Assign extracted values safely
            product_rating = rating.text.strip() if rating else "N/A"
            product_reviews = review_count.text.strip() if review_count else "N/A"
            material = material_info.text.strip() if material_info else "N/A"
            availability = "In Stock" if stock_info else "Out of Stock"
            seller = seller_info.text.strip() if seller_info else "N/A"

            # Append data
            purses.append([brand, name, discounted_price, original_price, product_rating, product_reviews, material, availability, seller, link])

        except Exception as e:
            print(f"⚠️ Error scraping product {link}: {e}")

    # Check if there is a next page
    try:
        next_button = driver.find_element(By.XPATH, "//a[@rel='next']")
        next_page_link = next_button.get_attribute("href")
        if next_page_link:
            driver.get(next_page_link)
            time.sleep(3)  # Wait for new page to load
            page += 1
        else:
            break
    except:
        print("🚫 No more pages to scrape.")
        break

# Close WebDriver
driver.quit()

# Convert to DataFrame
df = pd.DataFrame(purses, columns=["Brand", "Product Name", "Discounted Price", "Original Price", "Rating", "Reviews", "Material", "Availability", "Seller", "Product URL"])

# Save to CSV
df.to_csv("myntra_purses.csv", index=False)

print("✅ Scraping Complete! Data saved to 'myntra_purses.csv'")

📄 Scraping Page 1...
🔗 Opening Product: https://www.myntra.com/clutches/mochi/mochi-embellished-purse-clutch/31394659/buy
🔗 Opening Product: https://www.myntra.com/clutches/lavie/lavie-tiana-pink--white-embellished-potli-/17503298/buy
🔗 Opening Product: https://www.myntra.com/clutches/mango/mango-women--shoulder-strap-clutch/31238711/buy
🔗 Opening Product: https://www.myntra.com/clutches/miraggio/miraggio-black-gold-toned-corc-textured-shoulder-bag/16833704/buy
🔗 Opening Product: https://www.myntra.com/clutches/lino+perros/lino-perros-women-floral-embroidered-purse/24435832/buy
🔗 Opening Product: https://www.myntra.com/clutches/allen+solly/allen-solly-textured-embellished-purse-clutch/31306653/buy
🔗 Opening Product: https://www.myntra.com/clutches/allen+solly/allen-solly-embellished-party-purse-clutch/30908078/buy
🔗 Opening Product: https://www.myntra.com/clutches/anekaant/anekaant-gold--silver-embellished-faux-silk-purse-clutch/30397714/buy
🔗 Opening Product: https://www.myntra.com/cl

In [34]:
df

,Brand,Product Name,Discounted Price,Original Price,Rating,Reviews,Material,Availability,Seller,Product URL
0,Mochi,Embellished Purse Clutch,₹878,MRP ₹1690,4.5|29 Ratings,29 Ratings,N/A,In Stock,N/A,https://www.myntra.com/clutches/mochi/mochi-em...
1,Lavie,Tiana Pink & White Embellished Potli,₹647,MRP ₹2399,4.5|6.1k Ratings,6.1k Ratings,N/A,In Stock,N/A,https://www.myntra.com/clutches/lavie/lavie-ti...
2,MANGO,Women Shoulder Strap Clutch,₹2232,MRP ₹2790,N/A,N/A,N/A,In Stock,N/A,https://www.myntra.com/clutches/mango/mango-wo...
3,MIRAGGIO,Black Gold-Toned Corc Textured Shoulder Bag,₹1399,MRP ₹2499,4.6|1.1k Ratings,1.1k Ratings,N/A,In Stock,N/A,https://www.myntra.com/clutches/miraggio/mirag...
4,Lino Perros,Women Floral Embroidered Purse,₹1637,MRP ₹3995,4.4|206 Ratings,206 Ratings,N/A,In Stock,N/A,https://www.myntra.com/clutches/lino+perros/li...
5,Allen Solly,Textured Embellished Purse Clutch,₹1324,MRP ₹2499,4.9|15 Ratings,15 Ratings,N/A,In Stock,N/A,https://www.myntra.com/clutches/allen+solly/al...
6,Allen Solly,Embellished Party Purse Clutch,₹1218,MRP ₹2299,4.9|14 Ratings,14 Ratings,N/A,In Stock,N/A,https://www.myntra.com/clutches/allen+solly/al...
7,Anekaant,Gold & Silver Embellished Faux Silk Purse Clutch,₹2639,MRP ₹4399,4.2|44 Ratings,44 Ratings,N/A,In Stock,N/A,https://www.myntra.com/clutches/anekaant/aneka...
8,IRTH,Women Solid Purse,₹2097,MRP ₹3495,5|7 Ratings,7 Ratings,N/A,In Stock,N/A,https://www.myntra.com/clutches/irth/irth-wome...
9,Allen Solly,Quilted Purse Clutch with Sequins,₹1483,MRP ₹2799,N/A,N/A,N/A,In Stock,N/A,https://www.myntra.com/clutches/allen+solly/al...


In [36]:
replace_list = ["Ò","Ó",".","Õ","Ò","Ó","@ ","@. @. @","@\"","couldn't","can't","I'm","It's",]
to_be_replaced_list = ["\"","\"",". @", "\'", "\"", "\"", "@", "", "\"", "could not", "cannot", "I am", "It is"]

def amira_upgra():
    x=input("Enter the story:")
    for i in range(len(replace_list)):
        x=x.replace(replace_list[i], to_be_replaced_list[i])
    print(x)

amira_upgra()

[ CHAPTER 1 ] It was Sunday afternoon. @I was sitting on the edge of the comfy couch in the TV room of my house. @I'd been flipping the remote at lightning speed. @I was going back and forth between two channels that most people never even watch. @My friend Eddy's head was hanging upside down next to me. "Peter," she said. "Really. @Stop!" Yes, Eddy is a girl. @Her real name is Meredith Parker, but she took the "Ed" from the middle of "Meredith" and made it into "Eddy. " Eddy was sitting on the couch. @Her feet were hanging over the back. @Her head was where her legs should have been. @She rolled her eyes. "Can you please stop changing the channel for, like, half a second?" she said. @Eddy knows me pretty well. @We've been neighbors since we were babies. @We grew up together. @And she knows that I use the remote control faster and better than anyone. "Eddy," I replied, "Georgetown is playing Johns Hopkins on channel 122, and Cornell is playing Duke on 183!" I'd been switching between t